In [1]:
pip install flask

Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the 'C:\Users\Théo\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [15]:
import crepe
from scipy.io import wavfile
import numpy as np
import librosa
from midiutil import MIDIFile

In [93]:
sr, audio = wavfile.read('D:\Théo\MedleyDB\V1\AClassicEducation_NightOwl\AClassicEducation_NightOwl_STEMS\AClassicEducation_NightOwl_STEM_03.wav')
audio = audio[sr*60:sr*120]
time, frequency, confidence, activation = crepe.predict(audio, sr, viterbi=True)

188/188 [==============================] - 70s 370ms/step


In [127]:
confidence_list = []

for i,j in enumerate(confidence) :
        if j > 0.90 :
            confidence_list.append(frequency[i]) 
        else :
            continue

In [128]:
def convert_freq2midi_scalar(f, fA4InHz):

    if f <= 0:
        return 0
    else:
        return (69 + 12 * np.log2(f/fA4InHz))

def ToolFreq2Midi(fInHz, fA4InHz = 440):

    fInHz = np.asarray(fInHz)
    if fInHz.ndim == 0:
       return convert_freq2midi_scalar(fInHz,fA4InHz)

    midi = np.zeros(fInHz.shape)
    for k,f in enumerate(fInHz):
        midi[k] =  convert_freq2midi_scalar(f,fA4InHz)
            
    return (midi)

In [129]:
 Freq_midi = (ToolFreq2Midi(confidence_list, fA4InHz=440)).astype(int)

In [130]:
Freq_midi

array([65, 65, 65, ..., 63, 63, 63])

In [126]:
degrees  = Freq_midi # MIDI note number
track    = 0
channel  = 0
time     = 0    # In beats
duration = 1    # In beats
tempo    = tempo   # In BPM
volume   = 100  # 0-127, as per the MIDI standard

MyMIDI = MIDIFile(1)  # One track, defaults to format 1 (tempo track is created
                      # automatically)
MyMIDI.addTempo(track, time, tempo)

for i, pitch in enumerate(degrees):
    MyMIDI.addNote(track, channel, pitch, time + i, duration, volume)

with open("don't-know1.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)